# Problem Statment:¶
1. Loan Elegibility
2. LoanAmount
3. Loan Amount Term

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
import pandas as pd
train = pd.read_csv("/users/ravi/Desktop/datasets/training_set_loan.csv")
test = pd.read_csv("/users/ravi/Desktop/datasets/testing_set_loan.csv")

In [3]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,NaN,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


# Missing Data Treatment¶


In [5]:
def replacer(df):
    import pandas as pd
    Q = pd.DataFrame(df.isna().sum(),columns=["ct"])
    for i in Q[Q.ct > 0].index:
        if(df[i].dtypes == "object"):
            x = df[i].mode()[0]
            df[i] = df[i].fillna(x)
        else:
            x = df[i].mean()
            df[i] = df[i].fillna(x)
            

In [6]:
replacer(train)
replacer(test)

# Declare X and Y and Remove Unwanted

In [7]:
Y = train[["Loan_Status"]]
X = train.drop(labels=["Loan_ID","Loan_Status"],axis=1)

# Exploratory Data Analysis

In [8]:
def anova(df, cat, con):
    from statsmodels.formula.api import ols
    from statsmodels.stats.anova import anova_lm
    rel = con + "~" + cat
    model = ols(rel,df).fit()
    anova_result = anova_lm(model)
    Df = pd.DataFrame(anova_result)
    q = Df['PR(>F)'][cat]
    q = round(q,4)

In [9]:
for i in X.columns:
    if(X[i].dtypes != "object"):
        print(i,anova(train,"Loan_Status",i))
        print("------------------------------")
    else:
        print(pd.crosstab(train['Loan_Status'],train[i]))
        print("------------------------------")

Gender       Female  Male
Loan_Status              
N                37   155
Y                75   347
------------------------------
Married       No  Yes
Loan_Status          
N             79  113
Y            134  288
------------------------------
Dependents     0   1   2  3+
Loan_Status                 
N            113  36  25  18
Y            247  66  76  33
------------------------------
Education    Graduate  Not Graduate
Loan_Status                        
N                 140            52
Y                 340            82
------------------------------
Self_Employed   No  Yes
Loan_Status            
N              166   26
Y              366   56
------------------------------
ApplicantIncome None
------------------------------
CoapplicantIncome None
------------------------------
LoanAmount None
------------------------------
Loan_Amount_Term None
------------------------------
Credit_History None
------------------------------
Property_Area  Rural  Semiurban  Urban
L

# Preprocessing

In [10]:
from sklearn.preprocessing import StandardScaler
cat = []
con = []
for i in X.columns:
    if(X[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

ss = StandardScaler()
X1 = pd.DataFrame(ss.fit_transform(X[con]),columns=con)
X2 = pd.get_dummies(X[cat])

Xnew = X1.join(X2)

In [11]:
Xnew.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0.072651,-0.554368,0.000000,0.279851,0.45164,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1
1,0.000000,-0.038610,-0.219273,0.279851,0.45164,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
2,-0.394096,-0.554368,-0.957641,0.279851,0.45164,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1
3,-0.462412,0.252103,-0.314547,0.279851,0.45164,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1
4,0.097389,-0.554368,-0.064454,0.279851,0.45164,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1


# Decision Tree


In [12]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=31)

from sklearn.model_selection import GridSearchCV
tp = {"max_depth":range(2,20,1)}
cv = GridSearchCV(dtc,tp,scoring="accuracy",cv=3)
cvmodel = cv.fit(Xnew,Y)
x = cvmodel.best_params_['max_depth']

dtc = DecisionTreeClassifier(random_state=31,max_depth=x)
model = dtc.fit(Xnew,Y)

In [13]:
Xnew.shape


(614, 20)

# Data Preparation for test Data

In [14]:
test = test.drop(labels=["Loan_ID"],axis=1)

from sklearn.preprocessing import StandardScaler
cat = []
con = []
for i in test.columns:
    if(test[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

ss = StandardScaler()
X1 = pd.DataFrame(ss.fit_transform(test[con]),columns=con)
X2 = pd.get_dummies(test[cat])

Xpred = X1.join(X2)

# Check whether you have same columns in Xnew and Xpred

In [15]:
Xnew.shape

(614, 20)

In [16]:
Xpred.shape


(367, 20)

# Predictions

In [17]:
test["Predicted_Loan_Status"]=model.predict(Xpred)

# Problem_Statment 2: People whose loan got rejected, find how much can get approve

# Find out whose loan got rejected from test data

In [18]:
rejects = test[test.Predicted_Loan_Status == "N"]

# Prepare training Data

In [19]:
Xnew1 = test[test.Predicted_Loan_Status == "Y"]
Xnew2 = train[train.Loan_Status == "Y"]

In [20]:
Xnew1.shape

(303, 12)

In [21]:
Xnew2.shape

(422, 13)

In [22]:
Xnew1.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Predicted_Loan_Status'],
      dtype='object')

In [23]:
Xnew2.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [24]:
Xnew2 = Xnew2.drop(labels=["Loan_ID"],axis=1)

In [25]:
Xnew1.columns = Xnew2.columns

In [26]:
Xnew = pd.concat([Xnew1,Xnew2])

In [27]:
Xnew.shape

(725, 12)

In [28]:
Xnew.index = range(0,725,1)

In [29]:
Y = Xnew[["LoanAmount"]]
X = Xnew.drop(labels=["LoanAmount"],axis=1)

In [30]:
def preprocessing(df):
    import pandas as pd
    cat = []
    con = []
    for i in df.columns:
        if(df[i].dtypes == "object"):
            cat.append(i)
        else:
            con.append(i)
    X1 = pd.get_dummies(df[cat])
    from sklearn.preprocessing import StandardScaler
    ss = StandardScaler()
    X2 = pd.DataFrame(ss.fit_transform(df[con]),columns=con)
    X3 = X2.join(X1)
    return X3


In [31]:
X = preprocessing(X)

In [32]:
replacer(X)
replacer(Y)

# Regression model

In [33]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [34]:
dtr = DecisionTreeRegressor(random_state=21)
abr = AdaBoostRegressor(dtr,random_state=21,n_estimators=30)
model = abr.fit(X,Y)

# Predictions

In [35]:
rejects.drop(labels=["LoanAmount"],inplace=True,axis=1)
rejects.index = range(0,64,1)
replacer(rejects)
Xtest = preprocessing(rejects)
model.predict(Xtest)

array([ 72.        , 370.        , 168.        , 100.        ,
        74.        , 115.        ,  53.        , 175.        ,
       103.        , 120.        , 115.        , 137.        ,
       140.        , 185.        , 175.        ,  75.        ,
        50.        ,  98.        ,  47.        , 180.        ,
       113.        , 360.        , 259.        , 176.        ,
       150.        , 177.        ,  50.        ,  98.        ,
        65.        ,  48.        ,  98.        , 104.        ,
        98.        , 150.        , 108.        , 200.        ,
        90.        , 187.        , 146.41216216, 187.        ,
       185.        ,  47.        , 125.        , 296.        ,
       118.        ,  96.        , 114.        , 110.        ,
        98.        , 120.        , 254.        , 246.        ,
       115.        , 120.        , 120.        ,  47.        ,
       182.        , 106.        ,  48.        ,  97.        ,
       144.        , 142.        , 184.        , 157.  